In [1]:
#Importing libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O 
import os
import itertools
import warnings
warnings.filterwarnings("ignore")
import io

################# visualization #################
import matplotlib.pyplot as plt 
from PIL import  Image
import seaborn as sns 
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff

In [2]:
import pandas as pd
df = pd.read_csv (r"C:\Users\suvar\Desktop\Longistic regrassion\milknew.csv")

#first few rows
df.head()

,pH,Temprature,Taste,Odor,Fat,Turbidity,Colour,Grade
0,6.6,35,1,0,1,0,254,high
1,6.6,36,0,1,0,1,253,high
2,8.5,70,1,1,1,1,246,low
3,9.5,34,1,1,0,1,255,low
4,6.6,37,0,0,0,0,255,medium


# 2. Data Pre-Processing 

1. Data Type and conversion
2. Identifying & Treatment Missing Value 

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1059 entries, 0 to 1058
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   pH          1059 non-null   float64
 1   Temprature  1059 non-null   int64  
 2   Taste       1059 non-null   int64  
 3   Odor        1059 non-null   int64  
 4   Fat         1059 non-null   int64  
 5   Turbidity   1059 non-null   int64  
 6   Colour      1059 non-null   int64  
 7   Grade       1059 non-null   object 
dtypes: float64(1), int64(6), object(1)
memory usage: 66.3+ KB


In [4]:
df.isnull().sum()

pH            0
Temprature    0
Taste         0
Odor          0
Fat           0
Turbidity     0
Colour        0
Grade         0
dtype: int64

In [5]:
df.Odor.value_counts(dropna=False)

0    601
1    458
Name: Odor, dtype: int64

In [6]:
df.head()

,pH,Temprature,Taste,Odor,Fat,Turbidity,Colour,Grade
0,6.6,35,1,0,1,0,254,high
1,6.6,36,0,1,0,1,253,high
2,8.5,70,1,1,1,1,246,low
3,9.5,34,1,1,0,1,255,low
4,6.6,37,0,0,0,0,255,medium


# Data Visualization

#### Bad Rate Analysis 

In [8]:
df.Taste.value_counts()

1    585
0    474
Name: Taste, dtype: int64

In [9]:
(480/1059)*100

45.3257790368272

In [11]:
import plotly.express as px

fig = px.pie(df,names='Taste',color='Taste',
             color_discrete_map={0:'red',
                                 1:'green'})
fig.show()

### Reject customer data  = Bad Customer

In [12]:
df_Yes= df[df["Taste"] == 0]  # 269

df_Yes

,pH,Temprature,Taste,Odor,Fat,Turbidity,Colour,Grade
1,6.6,36,0,1,0,1,253,high
4,6.6,37,0,0,0,0,255,medium
11,5.6,50,0,1,1,1,255,low
12,8.6,55,0,1,1,1,255,low
14,6.8,45,0,1,1,1,255,high
...,...,...,...,...,...,...,...,...
1044,6.6,50,0,0,0,1,250,low
1045,6.5,36,0,0,0,0,247,medium
1046,6.6,38,0,0,0,0,255,medium
1049,6.5,37,0,0,0,0,255,medium


In [13]:
fig = px.sunburst(df_Yes, path=["pH","Temprature"])
fig.show()

# 5. Breaking Data into Two Part
 * Quantitative Variable
 * Qualitative Variable
 


In [14]:
df.select_dtypes(include=[np.number]).columns.tolist()

['pH', 'Temprature', 'Taste', 'Odor', 'Fat ', 'Turbidity', 'Colour']

In [15]:
# Qualitative Variable
Quantitative_Variable = df[df.select_dtypes(include=[np.number]).columns.tolist()]
Quantitative_Variable.head(3)

,pH,Temprature,Taste,Odor,Fat,Turbidity,Colour
0,6.6,35,1,0,1,0,254
1,6.6,36,0,1,0,1,253
2,8.5,70,1,1,1,1,246


In [16]:
Qualitative_Variable = df[df.select_dtypes(include=['object']).columns.tolist()]
Qualitative_Variable.head(3)

,Grade
0,high
1,high
2,low


### Converting qualitative Variable into number

In [17]:
from sklearn.preprocessing import LabelEncoder
Qualitative_Variable=Qualitative_Variable.apply(LabelEncoder().fit_transform) # label in ascending order
Qualitative_Variable.head()

,Grade
0,0
1,0
2,1
3,1
4,2


## Combining the data set 

In [18]:
Final_combined = pd.concat([Qualitative_Variable, Quantitative_Variable],axis=1)
Final_combined.head()

,Grade,pH,Temprature,Taste,Odor,Fat,Turbidity,Colour
0,0,6.6,35,1,0,1,0,254
1,0,6.6,36,0,1,0,1,253
2,1,8.5,70,1,1,1,1,246
3,1,9.5,34,1,1,0,1,255
4,2,6.6,37,0,0,0,0,255


# Data Partition 

In [19]:
from sklearn.model_selection import train_test_split

# define our input variable (X) & output variable
X = Final_combined.drop('Taste', axis = 1)
Y = Final_combined[['Taste']]
# Split X and y into X_
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=56)

# Model building 

# Hypothesis

* H0 :- There is no log linear relationship between Taste and all independent Variable

Vs 

* H1 :- There is log linear relationship between Taste and all independent Variable

Alpha = 0.05 (5%)

# Method 1 - Logistics Regression
#Needed to run the logistic regression
import statsmodels.formula.api as smf
result=smf.logit("Taste~C(Grade)+C(pH)+C(Temprature)+C(Odor)+C(Fat)+C(Turbidity)+C(Colour)+Age+Dependents+LoanAmount+Cibil_Score+ApplicantIncome",data=train).fit()
print(result.summary2())

# Method 2 - Logistics Regression

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logreg = LogisticRegression()
output=logreg.fit(X_train, y_train) # model

In [21]:
output.intercept_  # beto 

array([6.53081773])

In [22]:
output.coef_  # Beta 1 -- 

array([[-0.14644355, -0.01123506, -0.02099987, -0.87260834,  1.61556775,
         0.08228747, -0.02353013]])

In [23]:
output.n_features_in_

7

In [24]:
output.feature_names_in_

array(['Grade', 'pH', 'Temprature', 'Odor', 'Fat ', 'Turbidity', 'Colour'],
      dtype=object)

* We Reject Ho
* There is log linear relationship between Taste and Grade,pH,Temprature,Odor,Fat,Turbidity,Colour

# Step 6 : Predictions on Train Dataset


In [25]:
train=pd.concat([X_train,y_train],axis=1)
train.head()

,Grade,pH,Temprature,Odor,Fat,Turbidity,Colour,Taste
620,2,6.6,38,0,0,0,255,0
20,0,6.6,45,1,1,1,250,0
207,1,4.5,60,1,1,1,250,1
107,1,6.8,50,0,1,0,255,0
888,1,4.5,38,1,1,1,255,0


In [26]:
train['Predicted']=output.predict(X_train)
train.head()

,Grade,pH,Temprature,Odor,Fat,Turbidity,Colour,Taste,Predicted
620,2,6.6,38,0,0,0,255,0,0
20,0,6.6,45,1,1,1,250,0,1
207,1,4.5,60,1,1,1,250,1,1
107,1,6.8,50,0,1,0,255,0,1
888,1,4.5,38,1,1,1,255,0,1


train['Predicted']=np.where(train['Probability'] >= 0.7,1,0) 
train.head()

# Step 7 : Model Performance Metrics

In [27]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(train['Predicted'], train['Taste'])
matrix

array([[210,  85],
       [173, 379]], dtype=int64)

In [28]:
Accuracy_Train=((229+379)/(847)*100)
print(Accuracy_Train)

71.78276269185359


In [29]:
from sklearn.metrics import classification_report
print(classification_report(train['Taste'],train['Predicted']))

              precision    recall  f1-score   support

           0       0.71      0.55      0.62       383
           1       0.69      0.82      0.75       464

    accuracy                           0.70       847
   macro avg       0.70      0.68      0.68       847
weighted avg       0.70      0.70      0.69       847



* Accuracy of  Model is 70%,Hence Model is good Fit 
* Accuracy of Bad Customer Capture by Model is 82% ( Sensitivity )
* Accuracy of Good Customer Capture by Model is 55% (Specificity)

# Step 8 : Predictions on Test Dataset

In [30]:
test=pd.concat([X_test,y_test],axis=1)
test.head()

,Grade,pH,Temprature,Odor,Fat,Turbidity,Colour,Taste
847,1,6.5,37,1,1,1,245,0
931,0,6.8,45,1,1,0,245,1
764,1,8.6,55,1,1,1,255,0
1053,1,8.1,66,0,1,1,255,1
431,0,6.6,43,0,1,1,250,1


In [31]:
test['Predicted']=output.predict(X_test)
test.head()

,Grade,pH,Temprature,Odor,Fat,Turbidity,Colour,Taste,Predicted
847,1,6.5,37,1,1,1,245,0,1
931,0,6.8,45,1,1,0,245,1,1
764,1,8.6,55,1,1,1,255,0,0
1053,1,8.1,66,0,1,1,255,1,1
431,0,6.6,43,0,1,1,250,1,1


# Step 9 : Model Performance Metrics on Test data 

In [32]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(test['Predicted'],test['Taste'])
print(matrix)

[[61 28]
 [30 93]]


In [33]:
Accuracy_test=((61+93)/(212)*100)
Accuracy_test

72.64150943396226

In [34]:
from sklearn.metrics import classification_report
print(classification_report(test['Taste'],test['Predicted']))

              precision    recall  f1-score   support

           0       0.69      0.67      0.68        91
           1       0.76      0.77      0.76       121

    accuracy                           0.73       212
   macro avg       0.72      0.72      0.72       212
weighted avg       0.73      0.73      0.73       212



* Accuracy of  Model is 73%,Hence Model is good Fit 
* Accuracy of Bad Customer Capture by Model is 77% ( Sensitivity )
* Accuracy of Good Customer Capture by Model is 67% 

# Coefficient 

* For future Prediction below 4 variable data is mandatory 

* Note :- If anyone is not present output will not be reliable 

In [35]:
Model_Values=pd.DataFrame(output.intercept_,index=['Intercept'],columns=['Coefficient'])
Model_Values

,Coefficient
Intercept,6.530818


In [36]:
rows=["pH","Temprature","Odor","Fat","Turbidity","Colour","Grade"]

abc=pd.DataFrame(output.coef_.T,index=rows,columns=['Coefficient'])
abc

,Coefficient
pH,-0.146444
Temprature,-0.011235
Odor,-0.021000
Fat,-0.872608
Turbidity,1.615568
Colour,0.082287
Grade,-0.023530


In [37]:
Model_Values.append(abc)

,Coefficient
Intercept,6.530818
pH,-0.146444
Temprature,-0.011235
Odor,-0.021000
Fat,-0.872608
Turbidity,1.615568
Colour,0.082287
Grade,-0.023530


# Finish